In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
import numpy as np
import pickle
import random

from TicTacToe import Board, GameTools as gt

Initializing the neural network multiclassifier model.

This particular model has 3 hidden layers, with each layer containing 30 neurons. The activation function for the hidden layers is set to be ReLu.

In [ ]:
agent = MLPClassifier(hidden_layer_sizes=(30, 30, 30), activation='relu')
agent

Demonstrating how to use the partial_fit() method.

You must provide a matrix of features, an array of labels, and an array of all classes that your model may need to predict.

In [ ]:
features = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0]]) # 2d array since features are always in a matrix format
label = np.array([5])
classes = [1, 2, 3, 4, 5, 6, 7, 8, 9]

agent.partial_fit(X=features, y=label, classes=classes)
agent.loss_

Let's write a function that competes our agent against a random player.

In order to keep this demonstration as simple as possible, we will build this training environment with as few settings as possible. You may want to consider adding additional parameters so you can adjust how your model interacts with the environment.

In [ ]:
def environment(model: MLPClassifier, model_plays_first: bool, train_model: bool = False, print_board: bool = False):
    game = Board() # initializing a new game of TTT
    
    match_states = list() # a list to store the state of the board for each play made by our model
    match_labels = list() # a list to store the labels associated with each match state
    alt_labels = list() # a list to store alternative labels for the case where our model loses
    
    status = game.status() # function specific to TTT. Will be used to check if the game has ended
    
    moves_remaining = 9
    
    # while loop which plays a game of TTT
    while moves_remaining > 0 and not status[0]:
        state = deepcopy(game.vector) # getting copy of the board state in a vectorized format - Ex: [0, 0, 0, 0, 0, 0, 0, 0, 0]

        label = None
        
        coords = gt.avail_moves(game.board) # using a tool specific to our TTT game to get the (row, column) coordinates for available positions on the board

        matrix_state = np.array([state]) # putting our state into a numpy matrix. we need to use a consistent data structure and format for training the model and numpy works well
        
        # if model is playing first and it is first player's turn
        # this is the model's move
        if model_plays_first and moves_remaining % 2 != 0:
            prediction = model.predict(matrix_state)[0] # predicting the best move based on the current game board
            
            move = ((prediction - 1) // 3, (prediction - 1) % 3) # converting an integer into (row, column) coordinates. This is specific to our TTT
            
            # it is possible the model will try to make a move that isn't possible because the space is taken. We will validate that the move was made successfully here.
            out = game.move(row=move[0], column=move[1], player=1) # move() function returns True if move was successful, false otherwise
            
            if out:
                label = prediction
            else:
                move = random.choice(coords) # choosing randomly from known available positions
                coords.remove(move) # dropping our selected move from the available positions on the board
                
                label = (3 * move[0]) + move[1] + 1 # converting coordinates to integer value
                game.move(row=move[0], column=move[1], player=1) # placing random move on the board
                
            match_states.append(state) # saving state
            match_labels.append(label) # saving associated label
            
            if len(coords) > 0:
                move = random.choice(coords)
                pos = (3 * move[0]) + move[1] + 1
                alt_labels.append(pos) # saving alternate label
            else:
                alt_labels.append(label) # there was no alternative because all other positions were taken
        # if model is playing second and it is first player's turn
        # this is the random players move
        elif not model_plays_first and moves_remaining % 2 != 0:
            move = random.choice(coords)
            game.move(row=move[0], column=move[1], player=-1)
        
        # if model is playing first and it is the second player's turn
        # this is the random players move
        elif model_plays_first and moves_remaining % 2 == 0:
            move = random.choice(coords)
            game.move(row=move[0], column=move[1], player=-1)
            
        # if model is playing second and it is the second player's turn
        # this is the model's move
        else:
            prediction = model.predict(matrix_state)[0] # predicting the best move based on the current game board
            
            move = ((prediction - 1) // 3, (prediction - 1) % 3) # converting an integer into (row, column) coordinates. This is specific to our TTT
            
            # it is possible the model will try to make a move that isn't possible because the space is taken. We will validate that the move was made successfully here.
            out = game.move(row=move[0], column=move[1], player=1) # move() function returns True if move was successful, false otherwise
            
            if out:
                label = prediction
            else:
                move = random.choice(coords) # choosing randomly from known available positions
                label = (3 * move[0]) + move[1] + 1 # converting coordinates to integer value
                game.move(row=move[0], column=move[1], player=1) # placing random move on the board
                
            match_states.append(state) # saving state
            match_labels.append(label) # saving associated label
            
            if len(coords) > 0:
                move = random.choice(coords)
                pos = (3 * move[0]) + move[1] + 1
                alt_labels.append(pos) # saving alternate label
            else:
                alt_labels.append(label) # there was no alternative because all other positions were taken
        
        moves_remaining -= 1
        status = game.status()
    # end of the while loop for playing a game of TTT
    
    # if our setting for training the model is set to True, then the model will be partially fit to the data from this game
    # if set to false, this environment just allows a model to compete against a random player to evaluate performance
    if train_model:
        # converting our features, the match states, into a numpy matrix
        features = np.array(match_states)
        
        # converting our labels into a numpy array
        # if the model won, we use the winning labels
        # if the model lost, we use the alternate labels
        if status[1] != -1:
            labels = np.array(match_labels)
        else:
            labels = np.array(alt_labels)
        
        # a list explicitly stating the possible classes for our model - a required parameter for partial fitting
        classes = [x for x in range(1, 10)]
        
        # partially fitting the model to our data
        model.partial_fit(features, labels, classes)
    
    # if print_board set to true, will print a nice string formatting version of the board
    if print_board:
        print(game.print())
    
    return status[1], match_states, match_labels

Now we have a fully functional training environment for our agent.

We can run a single match without training and examine the output from our environment.

In [ ]:
t = environment(model=agent, model_plays_first=True, train_model=False, print_board=True)

It looks like our model is successfully playing a game of Tic Tac Toe! We can also see that our environment is able to successfully track all of the game states and decisions made by our model. This is good news!

Now, we can iteratively train the model and then see how it performs. In this case, we set the model to only play as the second player.

In [ ]:
# 10000 training iterations
for _ in range(10000):
    t = environment(model=agent, model_plays_first=False, train_model=True, print_board=False)

Our agent has now been trained on 10000 matches of TTT.

We can ask our agent to predict the best move from a random board as a test of its functionality.

In [ ]:
test_board = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0]])

agent.predict(test_board)

We can now put our agent into matches against a random player and evaluate how the agent performs. Notice that the "train_model" parameter is set to "False". Our agent will strictly play, it will not learn.

We will observe its performance when it plays first and when it plays second.

Anything other than a win is considered a loss.

In [ ]:
win_ct = 0
loss_ct = 0

for _ in range(5000):
    f = environment(model=agent, model_plays_first=True, train_model=False, print_board=False)
    
    if f[0] == 1:
        win_ct += 1
    else:
        loss_ct += 1
    
for _ in range(5000):
    s = environment(model=agent, model_plays_first=False, train_model=True, print_board=False)

    if s[0] == 1:
        win_ct += 1
    else:
        loss_ct += 1
        
win_ct, loss_ct

Lastly, we will want to save our agent as a binary file using the Pickle library. This will allow us to load our agent into whatever file we want for more training or to play the game.

In [ ]:
pickle.dump(agent, open("model_binaries/demo_agent.sav", "wb")) # saving the agent as a binary file

loaded_agent = pickle.load(open("model_binaries/demo_agent.sav", "rb"))
loaded_agent.predict(test_board)